In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

# Step 1: Load and Preprocess Datasets
def load_datasets():
    try:
        # File paths for datasets
        crops_file = 'final_karnataka_dataset.csv'
        nutrients_file = 'nutrients.csv'
        age_nutrients_file = 'age_wise_nutrients_consumtion_final.csv'
        
        # Load datasets
        crops_df = pd.read_csv(crops_file)
        nutrients_df = pd.read_csv(nutrients_file)
        age_nutrients_df = pd.read_csv(age_nutrients_file)
        
        # Standardize column names and string content to lowercase
        crops_df.columns = crops_df.columns.str.lower()
        nutrients_df.columns = nutrients_df.columns.str.lower()
        age_nutrients_df.columns = age_nutrients_df.columns.str.replace(r"\s*\(.*\)", "", regex=True).str.strip().str.lower()

        crops_df = crops_df.applymap(lambda x: x.lower() if isinstance(x, str) else x)
        nutrients_df = nutrients_df.applymap(lambda x: x.lower() if isinstance(x, str) else x)
        age_nutrients_df = age_nutrients_df.applymap(lambda x: x.lower() if isinstance(x, str) else x)
        
        print("Datasets loaded and preprocessed successfully!")
        return crops_df, nutrients_df, age_nutrients_df
    except Exception as e:
        print(f"Error loading datasets: {e}")
        return None, None, None

# Step 2: Analyze Nutrient Deficiency for All Districts
def analyze_all_districts(crops_df, nutrients_df, age_nutrients_df):
    # Get all unique districts from crops_df
    all_districts = crops_df['district_name'].str.lower().unique()
    all_results = {}

    for district_name in all_districts:
        print(f"\nProcessing District: {district_name.title()}")
        
        # Filter data for the selected district
        selected_district_crops = crops_df[crops_df['district_name'].str.lower() == district_name]
        if selected_district_crops.empty:
            print(f"No crop data available for district '{district_name}'. Skipping...")
            continue

        # Merge with nutrient data
        district_nutrient_data = selected_district_crops.merge(nutrients_df, on='crop', how='inner')
        nutrient_columns = ['carbohydrates', 'protein', 'fiber', 'fat', 'vitamin a', 'vitamin c', 
                            'iron', 'calcium', 'potassium', 'magnesium']
        nutrient_totals = district_nutrient_data[nutrient_columns].sum()

        # Calculate deficiencies for age groups
        deficiencies = {}
        for age_group in age_nutrients_df['group']:
            age_rdi = age_nutrients_df[age_nutrients_df['group'] == age_group][nutrient_columns]
            deficiency = age_rdi.values[0] - nutrient_totals.values
            deficiency_dict = {col: val for col, val in zip(nutrient_columns, deficiency) if val > 0}
            deficiencies[age_group] = deficiency_dict

        # Find the group with the highest deficiency
        high_deficiency_groups = {group: sum(def_dict.values()) for group, def_dict in deficiencies.items() if def_dict}
        if not high_deficiency_groups:
            print(f"No significant deficiencies found for '{district_name}'.")
            continue

        most_deficient_group = max(high_deficiency_groups, key=high_deficiency_groups.get)

        # Recommend crops for deficiencies
        recommended_crops = {}
        for nutrient, deficit in deficiencies[most_deficient_group].items():
            nutrient_rich_crops = nutrients_df[nutrients_df[nutrient] > 0].sort_values(by=nutrient, ascending=False)
            recommended_crops[nutrient] = nutrient_rich_crops[['crop', nutrient]].head(3).to_dict(orient='records')

        # Project future deficiencies using current trends (10-year growth)
        growth_rate = 0.05  # Annual growth rate of 5%
        projected_deficiencies = {
            nutrient: deficit * (1 + growth_rate) ** 10
            for nutrient, deficit in deficiencies[most_deficient_group].items()
        }

        # Find neighboring districts with surplus
        surplus_crops = {}
        neighboring_districts = crops_df[crops_df['state_name'] == selected_district_crops['state_name'].iloc[0]]['district_name'].unique().tolist()
        neighboring_districts.remove(district_name)
        for nutrient in deficiencies[most_deficient_group]:
            surplus_crops[nutrient] = []
            for neighbor in neighboring_districts:
                neighbor_crops = crops_df[crops_df['district_name'].str.lower() == neighbor.lower()]
                neighbor_data = neighbor_crops.merge(nutrients_df, on='crop', how='inner')
                if neighbor_data[nutrient].sum() > deficiencies[most_deficient_group][nutrient]:
                    surplus_crops[nutrient].append({
                        'district': neighbor,
                        'crops': neighbor_data[['crop', nutrient]].to_dict(orient='records'),
                        'surplus': neighbor_data[nutrient].sum()
                    })

        # Save results for the district
        all_results[district_name] = {
            "most_deficient_group": most_deficient_group,
            "deficiencies": deficiencies[most_deficient_group],
            "recommended_crops": recommended_crops,
            "projected_deficiencies": projected_deficiencies,
            "surplus_crops": surplus_crops
        }

    return all_results

# Load datasets
crops_df, nutrients_df, age_nutrients_df = load_datasets()

# Analyze all districts
if crops_df is not None and nutrients_df is not None and age_nutrients_df is not None:
    results = analyze_all_districts(crops_df, nutrients_df, age_nutrients_df)

    # Output results for all districts
    for district, data in results.items():
        print(f"\n=== District: {district.title()} ===")
        print(f"Most Deficient Group: {data['most_deficient_group']}")
        print("Deficient Nutrients:")
        for nutrient, amount in data['deficiencies'].items():
            print(f"- {nutrient}: {amount:.2f}")
        print("\nRecommended Crops:")
        for nutrient, crops in data['recommended_crops'].items():
            print(f"- {nutrient}:")
            for crop in crops:
                print(f"  * {crop['crop']} (Nutrient Value: {crop[nutrient]})")
        print("\n10-Year Projected Deficiencies:")
        for nutrient, future_value in data['projected_deficiencies'].items():
            print(f"- {nutrient}: {future_value:.2f}")
        print("\nSurplus Crops from Neighboring Districts:")
        for nutrient, surplus in data['surplus_crops'].items():
            print(f"- {nutrient}:")
            for district_surplus in surplus:
                print(f"  District: {district_surplus['district']}, Surplus: {district_surplus['surplus']}, Crops: {district_surplus['crops']}")
else:
    print("Dataset loading failed. Please ensure the files are available.")


/tmp/ipykernel_418771/2791215187.py:23: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  crops_df = crops_df.applymap(lambda x: x.lower() if isinstance(x, str) else x)
/tmp/ipykernel_418771/2791215187.py:24: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  nutrients_df = nutrients_df.applymap(lambda x: x.lower() if isinstance(x, str) else x)
/tmp/ipykernel_418771/2791215187.py:25: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  age_nutrients_df = age_nutrients_df.applymap(lambda x: x.lower() if isinstance(x, str) else x)


Datasets loaded and preprocessed successfully!

Processing District: Belgaum

Processing District: Bellary

Processing District: Bidar

Processing District: Chitradurga

Processing District: Dharwad

Processing District: Hassan

Processing District: Kolar

Processing District: Mandya

Processing District: Mysore

Processing District: Raichur

Processing District: Tumkur

=== District: Belgaum ===
Most Deficient Group: young male crossbred (1000 number)
Deficient Nutrients:
- carbohydrates: 109469395.80
- protein: 21030876.60
- fiber: 12495194.80
- fat: 25489162.30
- vitamin a: 328185769.00
- vitamin c: 29244874.70
- iron: 4397231.50
- calcium: 474499863.20
- potassium: 1715154198.00
- magnesium: 148154461.00

Recommended Crops:
- carbohydrates:
  * other fibres (Nutrient Value: 80.0)
  * ginger (Nutrient Value: 80.0)
  * jute (Nutrient Value: 76.0)
- protein:
  * soyabean (Nutrient Value: 36.0)
  * rapeseed & mustard (Nutrient Value: 26.0)
  * groundnut (Nutrient Value: 25.8)
- fiber:


In [3]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

def nutrient_exchange_policy(district_name, crops_df, nutrients_df, age_nutrients_df):
    # Step 1: Filter crop data for the selected district
    selected_district_crops = crops_df[crops_df['district_name'].str.lower() == district_name.lower()]
    if selected_district_crops.empty:
        print(f"No crop data found for the district: {district_name}")
        return None

    print(f"Crop data for district '{district_name}' loaded successfully.")
    
    # Step 2: Merge crop data with nutrient dataset and calculate total nutrients
    district_nutrient_data = selected_district_crops.merge(nutrients_df, on='crop', how='inner')
    nutrient_columns = ['carbohydrates', 'protein', 'fiber', 'fat', 'vitamin a', 'vitamin c',
                        'iron', 'calcium', 'potassium', 'magnesium']
    nutrient_totals = district_nutrient_data[nutrient_columns].sum()
    
    print(f"Total nutrients available in district '{district_name}':")
    print(nutrient_totals)

    # Step 3: Compare available nutrients with RDI to calculate deficiencies
    age_nutrients_df.columns = age_nutrients_df.columns.str.replace(r"\s*\(.*\)", "", regex=True).str.strip().str.lower()
    deficiencies = {}
    for age_group in age_nutrients_df['group']:
        age_rdi = age_nutrients_df[age_nutrients_df['group'] == age_group][nutrient_columns]
        deficiency = age_rdi.values[0] - nutrient_totals.values
        deficiency_dict = {col: val for col, val in zip(nutrient_columns, deficiency) if val > 0}
        deficiencies[age_group] = deficiency_dict

    # Identify most deficient group
    most_deficient_group = max(deficiencies, key=lambda g: sum(deficiencies[g].values()), default=None)
    if not most_deficient_group:
        print("No significant deficiencies found.")
        return None
    
    print(f"\nMost Deficient Group: {most_deficient_group}")
    print("Deficient Nutrients:")
    for nutrient, amount in deficiencies[most_deficient_group].items():
        print(f"- {nutrient}: {amount:.2f}")

    # Step 4: Recommend crops for the deficient nutrients
    print("\nSuggested Crops for Deficient Nutrients:")
    recommended_crops = {}
    for nutrient, deficit in deficiencies[most_deficient_group].items():
        nutrient_rich_crops = nutrients_df[nutrients_df[nutrient] > 0].sort_values(by=nutrient, ascending=False)
        top_crops = nutrient_rich_crops[['crop', nutrient]].head(3).to_dict(orient='records')  # Top 3 crops
        recommended_crops[nutrient] = top_crops
        print(f"- {nutrient}:")
        for crop in top_crops:
            print(f"  * {crop['crop']} (Nutrient Value: {crop[nutrient]})")

    # Step 5: Identify neighboring districts with surplus crops
    print("\nNeighboring Districts with Surplus Crops for Exchange:")
    surplus_crops = {}
    neighboring_districts = crops_df[crops_df['state_name'] == selected_district_crops['state_name'].iloc[0]]['district_name'].unique().tolist()
    neighboring_districts.remove(district_name)
    for nutrient in deficiencies[most_deficient_group]:
        surplus_crops[nutrient] = []
        for neighbor in neighboring_districts:
            neighbor_crops = crops_df[crops_df['district_name'].str.lower() == neighbor.lower()]
            neighbor_data = neighbor_crops.merge(nutrients_df, on='crop', how='inner')
            neighbor_surplus = neighbor_data[nutrient].sum()  # Get total nutrient value from neighboring district
            
            # Check if the surplus is ±30% of the deficiency
            lower_bound = 0.7 * deficiencies[most_deficient_group][nutrient]
            upper_bound = 1.3 * deficiencies[most_deficient_group][nutrient]
            if lower_bound <= neighbor_surplus <= upper_bound:
                surplus_crops[nutrient].append({
                    'district': neighbor,
                    'crops': neighbor_data[['crop', nutrient]].to_dict(orient='records'),
                    'surplus': neighbor_surplus
                })
    for nutrient, districts in surplus_crops.items():
        print(f"- {nutrient}:")
        for district in districts:
            print(f"  District: {district['district']}, Surplus: {district['surplus']}, Crops: {district['crops']}")

    # Step 6: Draft exchange policy
    print("\nExchange Policy for Nutrient Balance:")
    exchange_policy = {}
    for nutrient, districts in surplus_crops.items():
        if not districts:
            print(f"No surplus available for {nutrient}.")
            continue
        exchange_policy[nutrient] = []
        for district in districts:
            exchange_policy[nutrient].append({
                'receiving_district': district_name,
                'donor_district': district['district'],
                'crops_exchanged': district['crops'],
                'deficit_fulfilled': deficiencies[most_deficient_group][nutrient],
                'remaining_surplus': district['surplus'] - deficiencies[most_deficient_group][nutrient]
            })
    for nutrient, exchanges in exchange_policy.items():
        print(f"- {nutrient}:")
        for exchange in exchanges:
            print(f"  Receiving: {exchange['receiving_district']}, Donor: {exchange['donor_district']}")
            print(f"  Crops Exchanged: {exchange['crops_exchanged']}")
            print(f"  Deficit Fulfilled: {exchange['deficit_fulfilled']}")
            print(f"  Remaining Surplus: {exchange['remaining_surplus']}")

    return {
        "most_deficient_group": most_deficient_group,
        "deficiencies": deficiencies[most_deficient_group],
        "recommended_crops": recommended_crops,
        "surplus_crops": surplus_crops,
        "exchange_policy": exchange_policy
    }

# Example Usage
district_name = "chitradurga"  # Replace with the name of the district to analyze
results = nutrient_exchange_policy(district_name, crops_df, nutrients_df, age_nutrients_df)

# Output Summary
if results:
    print("\n=== Output Summary ===")
    print(f"Nutrient Deficiency for District: {district_name}")
    print(f"Most Deficient Group: {results['most_deficient_group']}")
    print("Deficient Nutrients:")
    for nutrient, amount in results['deficiencies'].items():
        print(f"- {nutrient}: {amount:.2f}")
    print("\nSuggested Crops for Deficient Nutrients:")
    for nutrient, crops in results['recommended_crops'].items():
        print(f"- {nutrient}:")
        for crop in crops:
            print(f"  * {crop['crop']} (Nutrient Value: {crop[nutrient]})")
    print("\nExchange Policy for Nutrient Balance:")
    for nutrient, exchanges in results['exchange_policy'].items():
        print(f"- {nutrient}:")
        for exchange in exchanges:
            print(f"  Receiving: {exchange['receiving_district']}, Donor: {exchange['donor_district']}")
            print(f"  Crops Exchanged: {exchange['crops_exchanged']}")
            print(f"  Deficit Fulfilled: {exchange['deficit_fulfilled']}")
            print(f"  Remaining Surplus: {exchange['remaining_surplus']}")


Crop data for district 'chitradurga' loaded successfully.
Total nutrients available in district 'chitradurga':
carbohydrates     29695.0
protein            8174.3
fiber              4033.9
fat                9922.6
vitamin a        101234.0
vitamin c          4081.2
iron               2527.4
calcium             126.2
potassium        312725.0
magnesium         86609.0
dtype: float64

Most Deficient Group: young male crossbred (1000 number)
Deficient Nutrients:
- carbohydrates: 109470305.00
- protein: 21031825.70
- fiber: 12495966.10
- fat: 25490077.40
- vitamin a: 328398766.00
- vitamin c: 29245918.80
- iron: 4397472.60
- calcium: 474499873.80
- potassium: 1715187275.00
- magnesium: 148163391.00

Suggested Crops for Deficient Nutrients:
- carbohydrates:
  * other fibres (Nutrient Value: 80.0)
  * ginger (Nutrient Value: 80.0)
  * jute (Nutrient Value: 76.0)
- protein:
  * soyabean (Nutrient Value: 36.0)
  * rapeseed & mustard (Nutrient Value: 26.0)
  * groundnut (Nutrient Value: 25.8)
